### MLOps using Kubeflow and Feature Store



#### Before start
- Google Cloud Storage Bucket / Pipelines / Feature Store must be created on same region {suggested: europe-west3(Frankfurt)}
- Pipelines considerations when failing if jobs is "in progress", needs to improve adding code until previous import is done
- Some issues with $tfx 2.7 so needed to downgrade to 2.5
- Align on aiplatform sdk
- Have good mlops use case for using feature store serving (costs)

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

In [ ]:
# %unload_ext nb_black
# # magic cmd to autoformat code
# #!pip install nb_black

# ! pip3 install {USER_FLAG} google-cloud-aiplatform==1.1.1
# ! pip3 install {USER_FLAG} google-cloud-pipeline-components==0.1.3
# ! pip3 install {USER_FLAG} --upgrade kfp
# ! pip3 install {USER_FLAG} numpy==1.20.3
# ! pip3 install --upgrade tensorflow
# ! pip3 install tf-nightly

##IMPORTS FOR ENVIRONMENT

In [ ]:
import os
import sys

from google_cloud_pipeline_components import aiplatform as gcc_aip

from kfp.v2 import compiler, dsl
from kfp.v2.google.client import AIPlatformClient
from google.cloud.aiplatform_v1 import (
    FeaturestoreOnlineServingServiceClient,
    FeaturestoreServiceClient,
)
from google.cloud.aiplatform_v1.types import FeatureSelector, IdMatcher
from google.cloud.aiplatform_v1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1.types import feature as feature_pb2
from google.cloud.aiplatform_v1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1.types import (
    featurestore_online_service as featurestore_online_service_pb2,
)
from google.cloud.aiplatform_v1.types import (
    featurestore_service as featurestore_service_pb2,
)
from google.cloud.aiplatform_v1.types import io as io_pb2
from google.protobuf.duration_pb2 import Duration
from google.cloud import aiplatform
from src.trainer import trainer_component
from src.generator import generator_component
from src.ingester import ingester_component
from src import load_component
import importlib
from src import feature_store_helper